## Spectral Radiance 
The spectral radiance is a radiometric wuantity that describes the amount of radiant energy emitted, reflected, transmitted, or received from a surface per unit solid angle, per unit projected area of the surface, and per unit wavelength or frequency interval.

**Planck's law** describes the spectral density of electromagnatic (EM) radiation $S(\lambda, T)$ of a certain wavelength $\lambda$ emitted by a black body in thermal equilibrium at a temperature $T$.  

$$ S(\lambda, T) = \frac{2 h c^2}{\lambda^5} \cdot \frac{1}{\exp\left(\tfrac{h c}{\lambda k T}\right) - 1} $$

- $S(\lambda, T)$: Spectral Radiance
- $h$: Planck constant,
- $c$: speed of light,
- $k$: Boltzmann constant,
- $T$: temperature (K),
- $\lambda$: wavelength (m)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import h, c, k, pi

def planck_lambda(lmbd, T):
    """
    Planck's law: spectral radiance per wavelength [W / (m^3 sr)]
    lmbd: wavelength in m
    T: temperature in K
    """
    return (2 * h * c**2 / lmbd**5) * 1.0 / (np.exp((h * c) / (lmbd * k * T)) - 1)

**Usage/Plot**

Below is an example usage of the "planck_lambda". It plots the spectral radiance for three different temperatures for a range of wavelengths in the infrared-(IR) Range.

In [ ]:
# Wavelengths from 0.01 to 14 micrometers, avoid zero to prevent division by zero, in meters
wavelength_um = np.linspace(0.01, 14, 1000)
wavelength_m = wavelength_um * 1e-6

temperatures = [300, 650, 1000]
labels = ["300 K", "650 K", "1000 K"]

plt.figure(figsize=(8, 5))
for T, label in zip(temperatures, labels):
    radiance = planck_lambda(wavelength_m, T)
    plt.plot(wavelength_um, radiance, label=label)

plt.xlabel("Wavelength (μm)")
plt.ylabel(r"Spectral Radiance $[W\,m^{-3}\,sr^{-1}]$")
plt.title("Spectral Radiance vs. Wavelength (Planck's Law)")
plt.yscale('log')
plt.ylim(1e6, 1e10)
plt.legend()
plt.xlim(0, 14)
plt.grid(True, which='both', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()
plt.show()

## Band Radiance
Planck's law can be integrated over a band to obtain the band radiance. This is the total radiant power emitted by a black body within that specific wavelength range.

$$ L(\lambda_1, \lambda_2, T) = \int_{\lambda_1}^{\lambda_2} S(\lambda, T) \, d\lambda $$

- $L(\lambda_1, \lambda_2, T)$: Band radiance
- $\lambda_1, \lambda_2$: Band borders
- $S(\lambda, T)$: Spectral radiance
- $\lambda$: wavelength (m)

In [ ]:
def integrate_band(T, lmbd_min, lmbd_max, n_samples=200):
    """
    Integrates Planck radiance over [lmbd_min, lmbd_max].
    Returns [W / (m^2 sr)].
    """
    wavelengths = np.linspace(lmbd_min, lmbd_max, n_samples)
    spectral = planck_lambda(wavelengths, T)
    return np.trapezoid(spectral, wavelengths)

## Atmospheric Transmission
When readiation passes through the atmosphere, it is attenuated. $\tau(\lambda)$ is the atmospheric transmission coefficient. It is simplified to be assummed constant for a small wavelength band $n$ (ex. MWIR/LWIR).
The formular for the measurable radiant power depends on the amount of times the EM-radiation passes through the atmosphere.

For a **one-way** path (ground -> satellite):
$$ L_{atm, ow}(n) = \tau(\lambda_n) \int_{\lambda_n}^{\lambda_{n+1}} S(\lambda, T) d\lambda $$
For a **two-way** path (sun -> ground -> satellite):
$$ L_{atm, tw}(n) = \tau(\lambda_n)^2 \int_{\lambda_n}^{\lambda_{n+1}} S(\lambda, T) d\lambda $$

- $L(n)$: Spectral Radiance (Measurable)
- $S(\lambda, T)$: Spectral Radiance
- $\tau(\lambda_n)$: Atmospheric transmission (constant for a band n)
- $\lambda_n$: Band Boarders

In [ ]:
def band_power_with_atmos(T, band_edges, tau_spectrum, two_way):
    """
    Calculates band-integrated power per band with atmospheric transmission.

    T: temperature [K]
    band_edges: array of wavelength edges [m]
    tau_spectrum: array of tau values for each band (len = len(band_edges)-1)

    Returns: array of band powers [W / (m^2 sr)]
    """
    results = []
    for i in range(len(band_edges)-1):
        l0, l1 = band_edges[i], band_edges[i+1]
        raw_band = integrate_band(T, l0, l1)
        if two_way:
            attenuated = (tau_spectrum[i]**2) * raw_band
        else: 
            attenuated = tau_spectrum[i] * raw_band
        results.append(attenuated)
    return np.array(results)

## Aperture Consideration
When calculating the radiative power from a pixel area on the ground $A_\mathrm{px}$ that reaches the detector through an optical system, the optical transmission $\tau_{opt}$, detector efficiency $\alpha_\mathrm{det}$ and the aperture angle $\theta_\mathrm{ap}$ must be considered. The aperture limits the solid angle over which radiation is collected.

The following integral computes the pixel power $P_\mathrm{px}$ received by the detector, integrating Planck’s spectral radiance $S_\lambda(\lambda, T)$ over wavelength and the solid angle $\theta$ defined by the aperture, not accounting for atmospheric transmission. The solid angle is completed by the angle $\phi$, resulting in a cone stating from the surface of the black body. 

$$
    P_\mathrm{px} = A_\mathrm{px} \cdot \tau_\mathrm{opt} \cdot \alpha_\mathrm{det} \cdot \int_{\lambda_\mathrm{min}}^{\lambda_\mathrm{max}} S(\lambda, T) d\lambda
    \int_{0}^{2\pi} 
    \int_{0}^{\theta_{ap}} \cos\theta \sin\theta d\theta d\phi
$$

- $P_{px}$: Pixel Power
- $A_\mathrm{px}$: Pixel area on the ground
- $\tau_\mathrm{opt}$: Optical transmission (simplified to be constant for a band)
- $\alpha_\mathrm{det}$: Detector efficiency (simplified to be constant for a band)
- $[\lambda_\mathrm{min}, \lambda_\mathrm{max}]$: Band edges
- $\theta_{ap}$: Aperture half-angle
- $\phi$: Cone angle

The integration over the aperture can be performed analytically. For a circular aperture with half-angle $\theta_{ap}$ and Lambertian emission, integration over the solid angle yields a constant factor $\pi \sin^2(\theta_\mathrm{ap})$. Substituting results in a simplified formular:

$$
P_\mathrm{px} = A_\mathrm{px} \cdot \tau_\mathrm{opt} \cdot \alpha_\mathrm{det} \cdot \left[ \int_{\lambda_\mathrm{min}}^{\lambda_\mathrm{max}} S(\lambda, T)\; d\lambda \right] \cdot \pi \sin^2(\theta_\mathrm{ap})
$$

- $P_{px}$: Pixel Power
- $A_\mathrm{px}$: Pixel area on the ground
- $\tau_\mathrm{opt}$: Optical transmission (simplified to be constant for a band)
- $\alpha_\mathrm{det}$: Detector efficiency (simplified to be constant for a band)
- $[\lambda_\mathrm{min}, \lambda_\mathrm{max}]$: Band edges
- $\theta_{ap}$: Aperture half-angle

In [ ]:
def pixel_power_aperture(T, lmbd_min, lmbd_max, theta_ap, A_px, tau_opt=0.9, alpha_det=0.9, n_samples=400):
    """
    Calculates pixel power received through an aperture.
    T: temperature [K]
    lmbd_min, lmbd_max: wavelength bounds [m]
    theta_ap: aperture half-angle [rad]
    A_px: pixel area [m^2]
    tau_opt: optical transmission
    alpha_det: detector efficiency
    """
    # Integrate Planck over wavelength
    wavelengths = np.linspace(lmbd_min, lmbd_max, n_samples)
    spectral = planck_lambda(wavelengths, T)
    I_band = np.trapz(spectral, wavelengths)  # [W / (m^2 sr)]

    # Integrate over solid angle defined by aperture
    Omega_ap = 2 * pi * (1 - np.cos(theta_ap))  # [sr]
    # For Lambertian source, radiance is weighted by cos(theta)
    solid_angle_factor = pi * (np.sin(theta_ap))**2  # ∫cosθ sinθ dθ dΦ over aperture

    # Total power
    P_px = A_px * tau_opt * alpha_det * I_band * solid_angle_factor
    return P_px

**Usage**

In [ ]:
pixel_power_aperture(T=300,
                     lmbd_min=1e-6, lmbd_max=2e-6,
                     theta_ap=np.radians(5), A_px=1e-4,
                     tau_opt=0.9, alpha_det=0.9)
# Example usage of the function
# This will calculate the pixel power received through an aperture
# at a temperature of 300 K, with specified wavelength bounds, aperture angle,
# pixel area, optical transmission, and detector efficiency.    

## Total Pixel-Power: Radiation, Albedo, Glint
The total power, reaching a pixel of a detector in space, looking at earth's surface is composed of three Components:

1. **Thermic emission of the Surface** $P_{surface}$

    The thermal radiation emitted by the Earth's surface itself due to its temperature.

2. **Reflected radiation from the sun (Albedo)** $P_(albedo)$

    The solar radiation incident on Earth's surface that is reflected back.

3. **Glint** $P_{glint}$

    A particular type of reflected sunlight characterized by a strong specular reflection, often from smooth water surfaces or other shiny surfaces, resulting in a bright reflection seen at a specific viewing angle.



The band radiance of each component is multiplied with the atmospheric and optical transmission coefficients $\tau_{n}$ and $\tau_{opt}$, the detector efficiency $\alpha_{det}$, the factor for the aperture half-angle $\pi \sin^2(\theta_\mathrm{ap})$ and the pixel area on the surface $A_{px}$:

$$
P_\mathrm{px} = P_\mathrm{surface} + P_\mathrm{albedo} + P_\mathrm{glint}
$$

When the optical transmission of a detector is assumed constant in the observable range (high-quality detector) and the atmospheric radiance is simplified to be constant for a band, the following formular describes the power reaching a pixel for a group of $n$ bands (ex. MWIR & LWIR), for a **one-way** path:

$$
P_\mathrm{Band,\,total,\,ow} = A_\mathrm{px} \cdot \tau_\mathrm{opt} \cdot \alpha_\mathrm{det} \cdot \left[ \sum_{n=1}^{N} \tau_n \cdot \int_{\lambda_n}^{\lambda_{n+1}} S_\lambda(\lambda, T)\, d\lambda \right]\cdot \pi \sin^2(\theta_\mathrm{ap}) 
$$

And for a **two-way** path:

$$
P_\mathrm{Band,\,total,\,tw} = A_\mathrm{px} \cdot \tau_\mathrm{opt} \cdot \alpha_\mathrm{det} \cdot \left[ \sum_{n=1}^{N} \tau_n^2 \cdot \int_{\lambda_n}^{\lambda_{n+1}} S_\lambda(\lambda, T)\, d\lambda  \right] \cdot \pi \sin^2(\theta_\mathrm{ap})
$$

- $P_{px}$: Pixel Power
- $A_\mathrm{px}$: Pixel area on the ground
- $\tau_\mathrm{n}$: Atmospheric Transmission (simplified to be constant for a band)
- $\tau_\mathrm{opt}$: Optical transmission (simplified to be constant for the measurable wavelengths)
- $\alpha_\mathrm{det}$: Detector efficiency (simplified to be constant for the measurable wavelengths)
- $[\lambda_\mathrm{n}, \lambda_\mathrm{n+1}]$: Band edges
- $\theta_{ap}$: Aperture half-angle
- $n$: Band number

**Components**
- **Surface Emission** 
    the one-way equation can be multiplied with the surface's emissivity $\epsilon_{ground}$

    $
    P_\mathrm{surface} = A_\mathrm{px} \cdot \tau_\mathrm{opt} \cdot \alpha_\mathrm{det} \cdot \epsilon_{ground} \cdot \left[ \sum_{n=1}^{N} \tau_n \cdot \int_{\lambda_n}^{\lambda_{n+1}} S_\lambda(\lambda, T)\, d\lambda \right]\cdot \pi \sin^2(\theta_\mathrm{ap}) 
    $

- **Albedo**
    In the worst canse scenario (the satellite and sun are in one line) the emission from the sun's surface is directly scattered by earth's surface. The radiation is calculated with Planck's law and the surface temperature of the sun, however a geometrical correction factor $G$ (projects the radiation of the sun's surface to a spherical surface with the radius of the distance between sun and earth) is introduced and multiplied with the universal two-way equation

    As most surfaces on earth are not only reflecting but also transmitting and absorbing, a reduction factor $\rho_{alb}$ is introduced.

    $G=\frac{r_{sun}^2}{d_{sun - earth}^2}$

    $P_\mathrm{albedo} = A_\mathrm{px} \cdot \tau_\mathrm{opt} \cdot \alpha_\mathrm{det} \cdot G \cdot \rho_{alb} \cdot \left[ \sum_{n=1}^{N} \tau_n^2 \cdot \int_{\lambda_n}^{\lambda_{n+1}} S_\lambda(\lambda, T_{sun})\, d\lambda  \right] \cdot \pi \sin^2(\theta_\mathrm{ap})$

- **Glint** 
    It is assumed that glint is a specular reflection of the sun on the surface of the earth and that it only reaches the imager when the sun is directly behind the imager. The effect is simplified to directly come from the sun's surface to the sensor ($d_{imager - earth} << d_{sun - earth}$). Again the radiation is calculated with Planck's law and the surface temperature of the sun.

    The largest surfaces that produce sun glint is water. The relation of reflection $\rho_{wat}$ is dependent on the refractive indixes of air $m_1$ and water $m_2$.

    $\rho_{wat} = frac{m_1 - m_2}{m_1 + m_2}^2$

    The relation of reflection and the ratio of the surface covered by $\delta_{glint}$ can now be multiplied with the universal formular for a two-way path.

    $P_\mathrm{Band,\,total,\,tw} = A_\mathrm{px} \cdot \tau_\mathrm{opt} \cdot \alpha_\mathrm{det} \cdot \delta_{glint} \cdot \rho_{wat} \cdot \left[ \sum_{n=1}^{N} \tau_n^2 \cdot \int_{\lambda_n}^{\lambda_{n+1}} S_\lambda(\lambda, T)\, d\lambda  \right] \cdot \pi \sin^2(\theta_\mathrm{ap})$
